# Functions (advanced)

[TOC]

We are going to cover the following topics:

- Decorators
- Lambdas
- Arguments and object's mutability
- Generators

As a refresher, we are going to revise one the most important concepts about functions in Python: **functions are objects**.

As an example, suppose that we want to create a "password checker", that is, a function that can verify if an input password complies with some rules (e.g., minumum length, a given number of special characters). We could create a function with the following signature:

```python
def check_password(password: str, min_length: int, min_uppercase: int, min_punctuation: int, min_digits: int) -> bool:
    """Check a given password again"""
```

The obvious problem is that different passwords might adhere to different rules. We would need to specify those rules for every password that should comply to those, and that's tedious! There must be a better way...

## Exercises

### Password checker `{functions as objects}`

Create a "password checker factory"

### Once per minute `{decorators}`

Create a decorator that checks how many times a function is run within a minute and restrict it to a maximum of once per minute

### String range `{generators}`

Create a generator function that emulates the the built-in `range`, but for characters

### Read `n` lines `{generators}`

Create a generator function that reads in the content of a file by `n` lines at a time